# 📘 Example: Measures and Strategies

In this notebook we will show you all the different options and inputs for creating different measures and how to create a strategy from the different measures. 

**Measures** in FloodAdapt can be categorized into **two** different categories, which we will cover all in this notebook:

**1. Hazard** - Mitigate the hazard directly

- Floodwall
- Pump
- Green Infrastructure

**2. Impact**  - Mitiage the impacts of the hazard on the assets

- Elevate properties 
- Buyout Properties
- Floodproof Properties

**Strategies** consist of multiple measures combined for example: ***StrategyXYZ* = *Floodwall + Elevated Homes + Pumps***

If you want to learn in more detail about measures and what a strategy is, please read the sections on [**Measures**](../../4_user_guide/strategy/index.qmd)  in the FloodAdapt GUI documentation.

## Import libraries

In [1]:
import pandas as pd
from pathlib import Path

from flood_adapt.flood_adapt import FloodAdapt
from flood_adapt.objects.measures.measures import *
from flood_adapt import Settings
from flood_adapt.objects.strategies.strategies import Strategy
from flood_adapt import unit_system as us

## 🚀 **Step 1**. Reading-in the FloodAdapt database

Let's start with initiating the database and FloodAdapt class. 
1. Initiate the database class [`Settings`](../../api_ref/Settings.qmd) by defining the `DATABASE_ROOT` and `DATABASE_NAME`.
2. Initiate the [`FloodAdapt`](../../api_ref/FloodAdapt.qmd) class by parsing the `Settings().database_path`.

In [2]:
# Define the static data folder
STATIC_DATA_DIR = Path("../../_data/examples/static-data/3_Measures").resolve()

# Set up the settings for the database
Settings(
    DATABASE_ROOT=Path("../../_data/examples").resolve(),
    DATABASE_NAME="charleston_test"
)

# Create the FloodAdapt instance
fa = FloodAdapt(Settings().database_path)

Attempt 1/5 failed to delete C:\Users\rautenba\repos\FloodAdapt\docs\_data\examples\charleston_test\output\scenarios\slr_pop_growth_seawall_water_square_elev_floodproof_build\Flooding\simulations\overland\sfincs_his.nc: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\rautenba\\repos\\FloodAdapt\\docs\\_data\\examples\\charleston_test\\output\\scenarios\\slr_pop_growth_seawall_water_square_elev_floodproof_build\\Flooding\\simulations\\overland\\sfincs_his.nc'
Attempt 2/5 failed to delete C:\Users\rautenba\repos\FloodAdapt\docs\_data\examples\charleston_test\output\scenarios\slr_pop_growth_seawall_water_square_elev_floodproof_build\Flooding\simulations\overland\sfincs_his.nc: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\rautenba\\repos\\FloodAdapt\\docs\\_data\\examples\\charleston_test\\output\\scenarios\\slr_pop_growth_seawall_water_square_elev_floodproof_build\\Floodi

## 🌧️ **Step 2**. Create Hazard Measures
We can divide the hazard measures into: 
1. **Hydraulic** (grey) measures -  structures such as levees, pumps, seawalls that protect infrastructure and residents
2. **Green infrastructure** measures - integrate natural elements like parks, green roofs, and permeable pavements into city

To create a hazard measure in Floodadapt we need to create a [`Measure`](../../api_ref/Measure.qmd) object. In the attributes we define which type of measure we want to apply using the `type` attribute. Depending on the type of measure different attributes must be parsed. 
There are **three** types of hazard measures:
1. [**Floodwall**](../../api_ref/objects/FloodWall.qmd) 
2. [**Pump**](../../api_ref/objects/Pump.qmd)
3. [**Green Infrastructure**](../../api_ref/objects/GreenInfrastructure.qmd)
    The green infrastructure class can be divided into **sub-categories**:  
    - Greening 
    - Total storage
    - Water square

Below we will create a `Measure` object for each of the three **hazard measure**.

### 🧱🌊 **Floodwall**
When we create a [`floodwall`](../../api_ref/objects/FloodWall.qmd) object we need to specify the `elevation` attribute to capture the height of the floodwall. To define the `elevation` we need to parse a [`UnitfulLength`]() object which consists of a `value` of type float and a `unit` which can be one of the [`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd). The `selection_type` describes the spatial type. This should be of `SelectionType.polyline` for a floodwall. 

In [3]:
# Create a measure object for a Floodwall
attrs_floodwall = FloodWall(
        name= "Seawall_12ft",
        description = "12ft Seawall",
        type = MeasureType.floodwall,
        selection_type = SelectionType.polyline,
        polygon_file = str(STATIC_DATA_DIR / "seawall.geojson"),
        elevation = us.UnitfulLength(value=12, units=us.UnitTypesLength.feet))

### ⛽💦 **Pump**
When we create a [`pump`](../../api_ref/objects/Pump.qmd) object we need to specify the `discharge` attribute to capture the total river discharge in the model. To define the `discharge` we need to parse a [`UnitfulLength`](../../api_ref/UnitfulLength.qmd) object which consists of a `value` of type float and a `unit` which can be one of the [`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd). The `selection_type` describes the spatial type. This should be of `SelectionType.polygon` for a pump. 

In [4]:
# Create a measure object for a Pump
attrs_pump = Pump(
        name= "Pump",
        description = "Pump",
        type = MeasureType.pump,
        selection_type = SelectionType.polygon,
        polygon_file = str(STATIC_DATA_DIR / "pump.geojson"),
        discharge = us.UnitfulDischarge(value=1, units=us.UnitTypesDischarge.cfs))

### 🌱🖼️ **Green infrastructure**
When we create a [`green infrastructure`](../../api_ref/objects/GreenInfrastructure.qmd) object we need to specify the `volume` attribute to capture the total storage of the green infrastructure. o define the `volume` we need to parse a [`UnitfulVolume`]() object which consists of a `value` of type float and a `unit` which can be one of the [`UnitTypesVolume`](../../api_ref/UnitTypesVolume.qmd). The `selection_type` describes the spatial type. This should be of `SelectionType.polygon` or `SelectionType.aggregation_area` for a green infrastructure. Other attributes like `height` and `percentage` are optional.  

In [5]:
# Create a measure object for Green infrastructure
attrs_greening = GreenInfrastructure(
        name= "green_infrastructure_storage",
        description = "Storage through green infrastructure",
        type = MeasureType.greening,
        selection_type = SelectionType.polygon,
        polygon_file = str(STATIC_DATA_DIR / "greening.geojson"),
        volume = us.UnitfulVolume(value = 43975190.31512848, units = us.UnitTypesVolume.cf),
        height = us.UnitfulHeight(value = 3, units = us.UnitTypesLength.feet),
        percent_area = 100.0)

attrs_total_storage = GreenInfrastructure(
        name=  "total_storage_aggregation_area",
        description = "Total Storage through green infrastructure in aggr area",
        type = MeasureType.total_storage,
        selection_type = SelectionType.aggregation_area,
        aggregation_area_type = "aggr_lvl_2",
        aggregation_area_name = "name5",
        volume = us.UnitfulVolume(value = 100000000.0, units = us.UnitTypesVolume.cf),
)

attrs_water_square = GreenInfrastructure(
        name=  "water_square",
        description = "Water Square",
        type = MeasureType.water_square,
        selection_type = SelectionType.polygon,
        polygon_file = str(STATIC_DATA_DIR / "water_square.geojson"),
        volume = us.UnitfulVolume(value =  43975190.31512848, units = us.UnitTypesVolume.cf),
        height = us.UnitfulHeight(value = 3, units = us.UnitTypesLength.feet))  

## 💾 **Step 3**. Saving the hazard measures to the database

In [6]:
# Save the measures to the database
fa.save_measure(attrs_floodwall)
fa.save_measure(attrs_pump)
fa.save_measure(attrs_greening)
fa.save_measure(attrs_total_storage)
fa.save_measure(attrs_water_square)

Let's have a look at some of the measures. We can for example explore the floodwall. If yo want to explore another measure, uncomment the code and re-run the cell below.

In [ ]:
gdf= gpd.read_file(STATIC_DATA_DIR / attrs_floodwall.polygon_file)
#gdf = gpd.read_file(STATIC_DATA_DIR/ attrs_pump.polygon_file)
#gdf = gpd.read_file(STATIC_DATA_DIR/ attrs_greening.polygon_file)
#gdf = gpd.read_file(STATIC_DATA_DIR/ attrs_water_square.polygon_file)
gdf.explore()

ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.

## 🏠 **Step 4:** Create Impact Measures
To create a impact measure in Floodadapt we need to create a [`Measure`](../../api_ref/Measure.qmd) object. In the attributes we define which type of measure we want to apply using the `type` attribute. Depending on the type of measure different attributes must be parsed. 
There are **three** types of impact measures:
1. [**Elevate**](../../api_ref/objects/Elevate.qmd)
2. [**Buy out**](../../api_ref/objects/BuyOut.qmd)
3. [**Floodproof**](../../api_ref/objects/FloodProof.qmd)

We can apply measures to a specific building occupancy by defining the occupancy in the `property_type` attribute. For example, if we only want to buyout **residential** homes we can parse the building type of the residential buildings here. Make sure you parse the same string-value as you use in your Delft-FIAT model to describe that type of building. 

If we want to apply the measure only in a specific aggregation area, we can define this with the `aggregation_area_type`, which describes the  name of the aggregation area category, and the `aggregation_area_name`, which responds to the name of the specific aggregation area within the category.

Below we will create an `Measure`-object for each **impact measure**.

### 🏠⬆️ **Elevate**
When we create a [`Elevate`](../../api_ref/objects/Elevate.qmd) object we need to specify the `elevation` attribute to capture the height of the elevation. To define the `elevation` we need to parse a [`UnitfulLengthRefValue`]() object which consists of a `value` of type float, a `unit` which can be one of the [`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd) and a vertical reference from which point the elevation should be calculated. This sholud be parsed as [`VerticalReference`]() object.

In [ ]:
# Create a measure object for elevating buildings
attrs_elevate = Elevate(
        name= "elevate_homes_2ft",
        description = "Elevate all residential buildings in aggregation area 1 by 2ft.",
        type = MeasureType.elevate_properties,
        selection_type = SelectionType.polygon,
        property_type = "residential",
        polygon_file = str(STATIC_DATA_DIR / "raise_property_polygon.geojson"),
        elevation = us.UnitfulLengthRefValue(value=2, units=us.UnitTypesLength.feet, type = us.VerticalReference.floodmap))

### 👥💰 **Buyout**
When we create a [`Buyout`](../../api_ref/objects/BuyOut.qmd) object we need to specify the `property_type` and either provide a spatial file for the area boundaries or define the `aggregation_area_type` and `aggregation_area_name`.

In [ ]:
# Create a measure object for buying out buildings
attrs_buyout = Buyout(
        name= "buyout_all_buildings",
        description = "Buyout all buildings in a specific area.",
        type = MeasureType.buyout_properties,
        selection_type = SelectionType.aggregation_area,
        aggregation_area_type = "aggr_lvl_2",
        aggregation_area_name = "name5",
        property_type = "ALL",
)

### 🏠🌊 **Floodproof**
When we create a [`FloodProof`](../../api_ref/objects/FloodProof.qmd) object we need to specify the `elevation` attribute to capture the height of the elevation. To define the `elevation` we need to parse a [`UnitfulLength`]() object which consists of a `value` of type float, a `unit` which can be one of the [`UnitTypesLength`](../../api_ref/UnitTypesLength.qmd).

In [ ]:
# Create a measure object for flood proofing buildings
attrs_flood_proof = FloodProof(
        name= "floodproof_all_com",
        description = "Floodproofing all commercial buildings.",
        type = MeasureType.floodproof_properties,
        selection_type = SelectionType.all,
        property_type = "commercial",
        elevation = us.UnitfulLength(value=2, units=us.UnitTypesLength.feet))

## 💾 **Step 5**. Saving the impact measures to the database

In [ ]:
# Save the measures to the database
fa.save_measure(attrs_elevate)
fa.save_measure(attrs_buyout)
fa.save_measure(attrs_flood_proof)

Let's have a look at some of the measures. We can for example explore the area in which all buildings will be elevated.

In [ ]:
gdf= gpd.read_file(Path(attrs_elevate.polygon_file))
gdf.explore()

Using the `get_measures()` method of the `FloodAdapt` class, we can check that the measures have been saved to the database.

In [ ]:
# Get a df with all strategies
pd.DataFrame(fa.get_measures())

## ✏️ **Step 6**: Copying and Editing a Measure in the database

If we want to edit small parts of a measure, it is easier to copy an existing measure and edit the copy. This way we do not have to create a new measure from scratch.

A measure can be copied in the database by using the `copy_measure()` method of the `FloodAdapt` class. This method takes three arguments: the name of the measure to be copied and the name and description of the new measure. Let's copy the measure we just created, having in mind that we want to make a measure for flood-proofing residential buildings.

In [ ]:
# Copy measure
fa.copy_measure(old_name="floodproof_all_com", new_name="floodproof_all_res", new_description="Floodproofing all residential buildings.")

We can see that now a new measure with name "floodproof_all_res" has been created in the database. However, the actual attributes of the measure are still the same as the original measure. 

In [ ]:
# Inspect Measure
floodproof_res = fa.get_measure("floodproof_all_res")
floodproof_res

We can directly edit the relevant attributes of the measure object. In this case, we want to change the type to "residential".

In [ ]:
# Edit attributes
floodproof_res.property_type = "residential"

Then using the `edit_measure()` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Measure` object. The `name` field of the measure object provided will be used to identify which measure is going to be updated in the database, with the given Measure object attributes.

In [ ]:
# Save updates
fa.save_measure(floodproof_res, overwrite = True)

Now we can verify that the measure has been updated in the database. The property type is now "residential".

In [ ]:
# Verify updates
floodproof_res = fa.get_measure("floodproof_all_res")
floodproof_res.property_type

## 🧩 **Step 6**. Create a Strategy
**Strategies** are combinations **measures**. They allow us to run an test multiple measures in a single model run. 

To create a strategy we need to create a [`Strategy`](../../api_ref/Strategy.qmd) object. In the `measures` attribute we parse a list of all the names of the measures that we want to apply in that strategy.

In [ ]:
# Create a strategy object
attrs_strategy = Strategy(
        name= "pump_greening_flood_proof",
        description = "Strategy with pump, greening and floodproofing",
        measures = [attrs_pump.name, attrs_greening.name, attrs_flood_proof.name],
        )

# Create the strategy
fa.create_strategy(attrs_strategy .model_dump(exclude_none=True))

# Save the strategy
fa.save_strategy(attrs_strategy )

Using the `get_strategies()` method of the `FloodAdapt` class, we can check that the strategies have been saved to the database.

In [ ]:
# Get a df with all strategies
pd.DataFrame(fa.get_strategies())

In [ ]:
# Inspect strategy
strategy = fa.get_strategy("pump_greening_flood_proof")
strategy

## ✏️ **Step 7**. Copying and Editing a Strategy in the database

If we want to edit small parts of a strategy, it is easier to copy an existing strategy and edit the copy. This way we do not have to create a new strategy from scratch.

A strategy can be copied in the database by using the `copy_strategy()` method of the `FloodAdapt` class. This method takes three arguments: the name of the strategy to be copied and the name and description of the new strategy. Let's copy the strategy we just created, having in mind that we want to remove the greening from the strategy and add a floodwall instead.

In [ ]:
# Copy strategy
fa.copy_strategy(old_name="pump_greening_flood_proof", new_name="pump_floodwall_flood_proof", new_description="Strategy with pump, flodwall and floodproofing.")

We can see that now a new strategy with name "pump_floodwall_flood_proof" has been created in the database. However, the actual attributes of the strtaegy are still the same as the original srtategy. 

In [ ]:
# Inspect strategy2
strategy_2 = fa.get_strategy("pump_floodwall_flood_proof")
strategy_2

We can directly edit the relevant attributes of the measure object. In this case, we want to change the type to "residential".

In [ ]:
# Update attributes
strategy_2.measures = strategy_2.measures.pop(attrs_greening.name)
strategy_2.measures = strategy_2.measures.append(attrs_floodwall.name)

Then using the `save_strategy(name, overwrite = True)` method of the `FloodAdapt` class, we can save the changes to the database. This method takes a single argument which is a `Strategy` object. The `name` field of the Strategy object provided will be used to identify which strategy is going to be updated in the database, with the given Strategy object attributes.

In [ ]:
# Save updates
fa.save_strategy("pump_floodwall_flood_proof", overwrtie = True)

Now we can verify that the strategy has been updated in the database.

In [ ]:
# Verify updates
strategy_2 = fa.get_strategy("pump_floodwall_flood_proof")
strategy_2.measures

## **Finished!** 
**Congratulations** you created all **measures** possible in FloodAdap and combined some of them into a **strategy**!